<a href="https://colab.research.google.com/github/SaiSJoshi/SaiSJoshi/blob/main/HW2P2_Starter_F22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW2P2: Face Classification and Verification


Congrats on coming to the second homework in 11785: Introduction to Deep Learning. This homework significantly longer and tougher than the previous homework. You have 2 sub-parts as outlined below. Please start early! 


*   Face Recognition: You will be writing your own CNN model to tackle the problem of classification, consisting of 7000 identities
*   Face Verification: You use the model trained for classification to evaluate the quality of its feature embeddings, by comparing the similarity of known and unknown identities

For this HW, you only have to write code to implement your model architecture. Everything else has been provided for you, on the pretext that most of your time will be used up in developing the suitable model architecture for achieving satisfactory performance.

Common errors which you may face in this homeworks (because of the size of the model)


*   CUDA Out of Memory (OOM): You can tackle this problem by (1) Reducing the batch size (2) Calling `torch.cuda.empty_cache()` and `gc.collect()` (3) Finally restarting the runtime



# Preliminaries

In [5]:
!nvidia-smi # to see what GPU you have

Thu Oct 20 03:00:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
!pip install wandb --quiet

In [7]:
import torch
import torch.nn as nn
from torchsummary import summary
import torchvision #This library is used for image-based operations (Augmentations)
import os
import gc
from tqdm import tqdm
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import glob
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [8]:
from google.colab import drive # Link your drive if you are a colab user
drive.mount('/content/drive') # Models in this HW take a long time to get trained and make sure to save it her

Mounted at /content/drive


# TODOs
As you go, please read the code and keep an eye out for TODOs!

# Download Data from Kaggle

In [9]:
# TODO: Use the same Kaggle code from HW1P2

!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"saisravanthijoshi","key":"707302665aa4b889e4e1f231d01d6ce5"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 3.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73275 sha256=09921de3efc373760678b6f1511d438e2ec7d10b0dd132ac642895daf861acf2
  Stored in directory: /root/.cache/pip/wheels/de/f7/d8/c3902cacb7e62cb611b1ad343d7cc07f42f7eb76ae3a52f3d1
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [10]:
# commands to download data from kaggle
# !kaggle competitions download -c 11-785-f22-hw1p2
!kaggle competitions download -c 11-785-f22-hw2p2-classification
!kaggle competitions download -c 11-785-f22-hw2p2-verification
!mkdir '/content/data'

!unzip -qo '11-785-f22-hw1p2.zip' -d '/content/data'
!mkdir '/content/data'

!unzip -qo '11-785-f22-hw1p2.zip' -d '/content/data'

 99% 2.35G/2.37G [00:11<00:00, 218MB/s]
100% 2.37G/2.37G [00:12<00:00, 211MB/s]
 54% 9.00M/16.8M [00:00<00:00, 73.3MB/s]
100% 16.8M/16.8M [00:00<00:00, 95.5MB/s]
unzip:  cannot find or open 11-785-f22-hw1p2.zip, 11-785-f22-hw1p2.zip.zip or 11-785-f22-hw1p2.zip.ZIP.
mkdir: cannot create directory ‘/content/data’: File exists
unzip:  cannot find or open 11-785-f22-hw1p2.zip, 11-785-f22-hw1p2.zip.zip or 11-785-f22-hw1p2.zip.ZIP.


In [11]:
!mkdir '/content/data'

!kaggle competitions download -c 11-785-f22-hw2p2-classification
!unzip -qo '11-785-f22-hw2p2-classification.zip' -d '/content/data'

!kaggle competitions download -c 11-785-f22-hw2p2-verification
!unzip -qo '11-785-f22-hw2p2-verification.zip' -d '/content/data'

mkdir: cannot create directory ‘/content/data’: File exists
11-785-f22-hw2p2-classification.zip: Skipping, found more recently modified local copy (use --force to force download)
11-785-f22-hw2p2-verification.zip: Skipping, found more recently modified local copy (use --force to force download)


# Configs

In [12]:
config = {
    'batch_size': 64, # Increase this if your GPU can handle it
    'lr': 0.1,
    'epochs': 50, # 10 epochs is recommended ONLY for the early submission - you will have to train for much longer typically.
    # Include other parameters as needed.
}

# Classification Dataset

In [13]:
DATA_DIR = '/content/data/11-785-f22-hw2p2-classification/'# TODO: Path where you have downloaded the data
TRAIN_DIR = os.path.join(DATA_DIR, "classification/train") 
VAL_DIR = os.path.join(DATA_DIR, "classification/dev")
TEST_DIR = os.path.join(DATA_DIR, "classification/test")

# Transforms using torchvision - Refer https://pytorch.org/vision/stable/transforms.html

train_transforms = torchvision.transforms.Compose([ 
    # Implementing the right transforms/augmentation methods is key to improving performance.
    #hor flip, rand aug
                    torchvision.transforms.RandomHorizontalFlip(0.5),
                    torchvision.transforms.RandAugment(),
                    torchvision.transforms.ColorJitter(brightness=0.1, contrast=0.1,saturation=0.1),
                    torchvision.transforms.ToTensor() 
                    ])
# Most torchvision transforms are done on PIL images. So you convert it into a tensor at the end with ToTensor()
# But there are some transforms which are performed after ToTensor() : e.g - Normalization
# Normalization Tip - Do not blindly use normalization that is not suitable for this dataset

val_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])


train_dataset = torchvision.datasets.ImageFolder(TRAIN_DIR, transform = train_transforms)
val_dataset = torchvision.datasets.ImageFolder(VAL_DIR, transform = val_transforms)
# You should NOT have data augmentation on the validation set. Why?


# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = config['batch_size'], 
                                           shuffle = True,num_workers = 4, pin_memory = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = config['batch_size'], 
                                         shuffle = False, num_workers = 2)

In [14]:
# You can do this with ImageFolder as well, but it requires some tweaking
class ClassificationTestDataset(torch.utils.data.Dataset):

    def __init__(self, data_dir, transforms):
        self.data_dir   = data_dir
        self.transforms = transforms

        # This one-liner basically generates a sorted list of full paths to each image in the test directory
        self.img_paths  = list(map(lambda fname: os.path.join(self.data_dir, fname), sorted(os.listdir(self.data_dir))))

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        return self.transforms(Image.open(self.img_paths[idx]))

In [15]:
test_dataset = ClassificationTestDataset(TEST_DIR, transforms = val_transforms) #Why are we using val_transforms for Test Data?
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = config['batch_size'], shuffle = False,
                         drop_last = False, num_workers = 2)

In [16]:
print("Number of classes: ", len(train_dataset.classes))
print("No. of train images: ", train_dataset.__len__())
print("Shape of image: ", train_dataset[0][0].shape)
print("Batch size: ", config['batch_size'])
print("Train batches: ", train_loader.__len__())
print("Val batches: ", val_loader.__len__())

Number of classes:  7000
No. of train images:  140000
Shape of image:  torch.Size([3, 224, 224])
Batch size:  64
Train batches:  2188
Val batches:  547


# ResNet

In [17]:
# class ResBlock(torch.nn.Module):
#     def __init__(self, in_channels, out_channels, downsample):
#         super().__init__()
#         if downsample:
#             self.conv1 = torch.nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
#             self.shortcut = torch.nn.Sequential(
#                 torch.nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
#                 torch.nn.BatchNorm2d(out_channels)
#             )
#         else:
#             self.conv1 = torch.nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
#             self.shortcut = torch.nn.Sequential()

#         self.conv2 = torch.nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
#         self.bn1 = torch.nn.BatchNorm2d(out_channels)
#         self.bn2 = torch.nn.BatchNorm2d(out_channels)

#     def forward(self, input):
#         shortcut = self.shortcut(input)
#         input = torch.nn.ReLU()(self.bn1(self.conv1(input)))
#         input = torch.nn.ReLU()(self.bn2(self.conv2(input)))
#         input = input + shortcut
#         return torch.nn.ReLU()(input)


In [18]:
# class Network(torch.nn.Module):
#     """
#     The Very Low early deadline architecture is a 4-layer CNN.

#     The first Conv layer has 64 channels, kernel size 7, and stride 4.
#     The next three have 128, 256, and 512 channels. Each have kernel size 3 and stride 2.
    
#     Think about strided convolutions from the lecture, as convolutioin with stride= 1 and downsampling.
#     For stride 1 convolution, what padding do you need for preserving the spatial resolution? 
#     (Hint => padding = kernel_size // 2) - Why?)

#     Each Conv layer is accompanied by a Batchnorm and ReLU layer.
#     Finally, you want to average pool over the spatial dimensions to reduce them to 1 x 1. Use AdaptiveAvgPool2d.
#     Then, remove (Flatten?) these trivial 1x1 dimensions away.
#     Look through https://pytorch.org/docs/stable/nn.html 
    
#     TODO: Fill out the model definition below! 

#     Why does a very simple network have 4 convolutions?
#     Input images are 224x224. Note that each of these convolutions downsample.
#     Downsampling 2x effectively doubles the receptive field, increasing the spatial
#     region each pixel extracts features from. Downsampling 32x is standard
#     for most image models.

#     Why does a very simple network have high channel sizes?
#     Every time you downsample 2x, you do 4x less computation (at same channel size).
#     To maintain the same level of computation, you 2x increase # of channels, which 
#     increases computation by 4x. So, balances out to same computation.
#     Another intuition is - as you downsample, you lose spatial information. We want
#     to preserve some of it in the channel dimension.
#     """

#     def __init__(self, num_classes=7000):
#         super().__init__()

#         kernel_size_l1 = 7
#         input_ch_l1 = 64
#         output_ch_l2 = 128
#         stride_l1 = 4

#         kernel_size_l2 = 3
#         output_ch_l3 = 256
#         stride_l2 = 2

#         kernel_size_l3 = 3
#         output_ch_l4 = 512
#         stride_l3 = 2
        


#         self.backbone = torch.nn.Sequential(
#                 # TODO

#                 torch.nn.Conv2d(3,64, stride=stride_l1,kernel_size=kernel_size_l1,padding=kernel_size_l1 // 2),
#                 torch.nn.BatchNorm2d(input_ch_l1),          
#                 torch.nn.ReLU(),

#                 torch.nn.AdaptiveAvgPool2d((1,1)), 
#                 torch.nn.Flatten(),
#         ) 
#         self.layer1 = torch.nn.Sequential(
#             ResBlock(64, 64, downsample=False),
#             ResBlock(64, 64, downsample=False),
#             ResBlock(64, 464, downsample=False)
#         )

#         self.layer2 = torch.nn.Sequential(
#             ResBlock(464,545, downsample=True),
#             ResBlock(545,545, downsample=False),
#             ResBlock(545,545, downsample=False),
#             ResBlock(545,545, downsample=False)
#         )

#         self.layer3 = torch.nn.Sequential(
#             ResBlock(545,1256, downsample=True),
#             ResBlock(1256, 2256, downsample=False),
#             ResBlock(2256, 2256, downsample=False),
#             ResBlock(2256, 2256, downsample=False),
#             ResBlock(4256, 2256, downsample=False),
#             ResBlock(2256, 2256, downsample=False)
#         )


#         self.layer4 = torch.nn.Sequential(
#             ResBlock(2256, 2512, downsample=True),
#             ResBlock(2512, 2512, downsample=False),
#             ResBlock(2512, 2512, downsample=False),
#         )
#         self.cls_layer = torch.nn.Linear(64,num_classes)
#         # self.cls_layer = torch.nn.Softmax()
#         #Is this classification layer?
    
#     def forward(self, x, return_feats=False):
#         """
#         What is return_feats? It essentially returns the second-to-last-layer
#         features of a given image. It's a "feature encoding" of the input image,
#         and you can use it for the verification task. You would use the outputs
#         of the final classification layer for the classification task.

#         You might also find that the classification outputs are sometimes better
#         for verification too - try both.
#         """
#         # print(x.shape)
#         feats = self.backbone(x)
#         out = self.cls_layer(feats)

#         if return_feats:
#             return feats
#         else:
#             return out
# print(Network())            
# model = Network().to(device)

# summary(model, (3, 224, 224))

# ConvNexT

In [19]:
class ConvNextBlock(torch.nn.Module):
    r""" ConvNeXt Block. There are two equivalent implementations:
    (1) DwConv -> LayerNorm (channels_first) -> 1x1 Conv -> GELU -> 1x1 Conv; all in (N, C, H, W)
    (2) DwConv -> Permute to (N, H, W, C); LayerNorm (channels_last) -> Linear -> GELU -> Linear; Permute back
    We use (2) as we find it slightly faster in PyTorch
    
    Args:
        dim (int): Number of input channels.
        drop_path (float): Stochastic depth rate. Default: 0.0
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
    """
    def __init__(self, dim):
        super().__init__()
       

        self.block = nn.Sequential(
            nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim),
            nn.BatchNorm2d(dim, eps=1e-6),
            nn.Conv2d(dim, 4* dim, kernel_size=1),
            nn.GELU(),
            nn.Conv2d(4*dim, dim, kernel_size=1)
         
r
        )
        
    def forward(self, x):
        return x+self.block(x)


model = ConvNextBlock(3).to(device)

summary(model, (3, 224, 224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 3, 224, 224]             150
       BatchNorm2d-2          [-1, 3, 224, 224]               6
            Conv2d-3         [-1, 12, 224, 224]              48
              GELU-4         [-1, 12, 224, 224]               0
            Conv2d-5          [-1, 3, 224, 224]              39
Total params: 243
Trainable params: 243
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 12.63
Params size (MB): 0.00
Estimated Total Size (MB): 13.21
----------------------------------------------------------------


In [20]:
class ConvNext(torch.nn.Module):

  def __init__(self, in_channels, out_channels):

    super().__init__()

    self.stem = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels, 96, kernel_size = 4, stride = 4),
        torch.nn.BatchNorm2d(96),
       )
    self.backbone = torch.nn.Sequential(
        ConvNextBlock(96),
        ConvNextBlock(96),
        ConvNextBlock(96),
        
        torch.nn. BatchNorm2d (96) ,
        torch.nn. Conv2d (96, 192, kernel_size= 2, stride= 2),

        ConvNextBlock(192),
        ConvNextBlock(192),
        ConvNextBlock(192),

        torch.nn.BatchNorm2d(192),
        torch.nn.Conv2d (192, 384, kernel_size= 2, stride= 2),

        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),
        ConvNextBlock(384),

        torch.nn.BatchNorm2d (384),
        torch.nn. Conv2d(384, 768, kernel_size= 2, stride= 2),

        ConvNextBlock(768),
        ConvNextBlock(768),
        ConvNextBlock(768),
        
        torch.nn.AdaptiveAvgPool2d(1),
        torch.nn.Flatten()

        
    )

    self.cls_layer = torch.nn.Linear(768,7000)

  def forward(self, x, return_feats = False):
    out = self.stem(x)
    out = self.backbone(out)

    if return_feats:
      return out

    return self.cls_layer(out)


          
model = ConvNext(3,7000).to(device)

summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 56, 56]           4,704
       BatchNorm2d-2           [-1, 96, 56, 56]             192
            Conv2d-3           [-1, 96, 56, 56]           4,800
       BatchNorm2d-4           [-1, 96, 56, 56]             192
            Conv2d-5          [-1, 384, 56, 56]          37,248
              GELU-6          [-1, 384, 56, 56]               0
            Conv2d-7           [-1, 96, 56, 56]          36,960
     ConvNextBlock-8           [-1, 96, 56, 56]               0
            Conv2d-9           [-1, 96, 56, 56]           4,800
      BatchNorm2d-10           [-1, 96, 56, 56]             192
           Conv2d-11          [-1, 384, 56, 56]          37,248
             GELU-12          [-1, 384, 56, 56]               0
           Conv2d-13           [-1, 96, 56, 56]          36,960
    ConvNextBlock-14           [-1, 96,

# Setup everything for training

In [21]:
# TODO: What loss do you need for a multi class classification problem?
criterion =torch.nn.CrossEntropyLoss() # label smoothing
optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9, weight_decay=1e-4)

# TODO: Implement a scheduler (Optional but Highly Recommended)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=4, threshold=0.001, threshold_mode='rel', cooldown=0, min_lr=1e-5, eps=1e-08, verbose=True)
# should read more than 60 with 0.1 learning rate

# You can try ReduceLRonPlateau, StepLR, MultistepLR, CosineAnnealing, etc.
scaler = torch.cuda.amp.GradScaler() # Good news. We have FP16 (Mixed precision training) implemented for you
# It is useful only in the case of compatible GPUs such as T4/V100

In [22]:
# #Load model
# best_model = torh.load(saved_model)

# Let's train!

In [23]:
def train(model, dataloader, optimizer, criterion):
    
    model.train()

    # Progress Bar 
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 
    
    num_correct = 0
    total_loss = 0

    for i, (images, labels) in enumerate(dataloader):
        
        optimizer.zero_grad() # Zero gradients

        images, labels = images.to(device), labels.to(device)
        
        with torch.cuda.amp.autocast(): # This implements mixed precision. Thats it! 
            outputs = model(images)
            loss = criterion(outputs, labels)

        # Update no. of correct predictions & loss as we iterate
        num_correct += int((torch.argmax(outputs, axis=1) == labels).sum())
        total_loss += float(loss.item())

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / (config['batch_size']*(i + 1))),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() 
        
        # TODO? Depending on your choice of scheduler,
        # You may want to call some schdulers inside the train function. What are these?
      
      
        batch_bar.update() # Update tqdm bar
    

    batch_bar.close() # You need this to close the tqdm bar

    acc = 100 * num_correct / (config['batch_size']* len(dataloader))
    total_loss = float(total_loss / len(dataloader))
    
    return acc, total_loss

In [24]:
def validate(model, dataloader, criterion):
  
    model.eval()
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)

    num_correct = 0.0
    total_loss = 0.0

    for i, (images, labels) in enumerate(dataloader):
        
        # Move images to device
        images, labels = images.to(device), labels.to(device)
        
        # Get model outputs
        with torch.inference_mode():
            outputs = model(images)
            loss = criterion(outputs, labels)

        num_correct += int((torch.argmax(outputs, axis=1) == labels).sum())
        total_loss += float(loss.item())

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / (config['batch_size']*(i + 1))),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct)

        batch_bar.update()

    batch_bar.close()
    acc = 100 * num_correct / (config['batch_size']* len(dataloader))
    total_loss = float(total_loss / len(dataloader))
    return acc, total_loss

In [25]:
gc.collect() # These commands help you when you face CUDA OOM error
torch.cuda.empty_cache()

# Wandb

In [26]:
wandb.login(key="10b3693ed9ef75468a46ff13fb32dc459cc8a099") #API Key is in your wandb account, under settings (wandb.ai/settings)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [27]:
# Create your wandb run
run = wandb.init(
    name = "ConvNext-trasform", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw2p2-ablations", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

wandb: Currently logged in as: saisravjo (stack-it). Use `wandb login --relogin` to force relogin


# Experiments

In [28]:
best_valacc = 0.0

for epoch in range(config['epochs']):

    curr_lr = float(optimizer.param_groups[0]['lr'])

    train_acc, train_loss = train(model, train_loader, optimizer, criterion)
    
    print("\nEpoch {}/{}: \nTrain Acc {:.04f}%\t Train Loss {:.04f}\t Learning Rate {:.04f}".format(
        epoch + 1,
        config['epochs'],
        train_acc,
        train_loss,
        curr_lr))
    
    val_acc, val_loss = validate(model, val_loader, criterion)
    scheduler.step(val_acc)
    print("Val Acc {:.04f}%\t Val Loss {:.04f}".format(val_acc, val_loss))

    wandb.log({"train_loss":train_loss, 'train_Acc': train_acc, 'validation_Acc':val_acc, 
               'validation_loss': val_loss, "learning_Rate": curr_lr})
    
    # If you are using a scheduler in your train function within your iteration loop, you may want to log
    # your learning rate differently 

    # #Save model in drive location if val_acc is better than best recorded val_acc
    if val_acc >= best_valacc:
      #path = os.path.join(root, model_directory, 'checkpoint' + '.pth')
      print("Saving model")
      torch.save({'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  'scheduler_state_dict':scheduler.state_dict(),
                  'val_acc': val_acc, 
                  'epoch': epoch}, './checkpoint.pth')
      best_valacc = val_acc
      wandb.save('checkpoint.pth')
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch 1/50: 
Train Acc 0.1393%	 Train Loss 8.5015	 Learning Rate 0.1000


Val Acc 0.6141%	 Val Loss 7.8036
Saving model



Epoch 2/50: 
Train Acc 2.1502%	 Train Loss 7.0983	 Learning Rate 0.1000


Val Acc 5.3931%	 Val Loss 6.2536
Saving model



Epoch 3/50: 
Train Acc 10.3041%	 Train Loss 5.5878	 Learning Rate 0.1000


...


Epoch 49/50: 
Train Acc 99.7929%	 Train Loss 0.0280	 Learning Rate 0.0100


Val Acc 84.4921%	 Val Loss 1.1458



Epoch 50/50: 
Train Acc 99.7972%	 Train Loss 0.0280	 Learning Rate 0.0100


Val Acc 84.4893%	 Val Loss 1.1304


learning_Rate,████████████████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_Acc,▁▁▂▃▄▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇████████████████████
train_loss,█▇▆▅▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_Acc,▁▁▂▃▅▅▆▆▆▆▆▆▆▆▆▇▆▆▇▆████████████████████
validation_loss,█▆▅▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_Rate,0.01
train_Acc,99.79719
train_loss,0.028
validation_Acc,84.48926
validation_loss,1.13039


# Classification Task: Testing

In [36]:
def test(model,dataloader):

  model.eval()
  batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='test')
  test_results = []
  
  for i, (images) in enumerate(dataloader):
      # TODO: Finish predicting on the test set.
      images = images.to(device)

      with torch.inference_mode():
        outputs = model(images)

      outputs = torch.argmax(outputs, axis=1).detach().cpu().numpy().tolist()
      test_results.extend(outputs)
      
      batch_bar.update()
      
  batch_bar.close()
  return test_results

In [37]:
test_results = test(model, test_loader)

## Generate csv to submit to Kaggle

In [38]:
with open("classification_early_submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(test_dataset)):
        f.write("{},{}\n".format(str(i).zfill(6) + ".jpg", test_results[i]))

# Verification Task: Validation

The verification task consists of the following generalized scenario:
- You are given X unknown identitites 
- You are given Y known identitites
- Your goal is to match X unknown identities to Y known identities.

We have given you a verification dataset, that consists of 1000 known identities, and 1000 unknown identities. The 1000 unknown identities are split into dev (200) and test (800). Your goal is to compare the unknown identities to the 1000 known identities and assign an identity to each image from the set of unknown identities. 

Your will use/finetune your model trained for classification to compare images between known and unknown identities using a similarity metric and assign labels to the unknown identities. 

This will judge your model's performance in terms of the quality of embeddings/features it generates on images/faces it has never seen during training for classification.

In [39]:
known_regex = "/content/data/verification/known/*/*"
known_paths = [i.split('/')[-2] for i in sorted(glob.glob(known_regex))] 
# This obtains the list of known identities from the known folder


#Change the directory accordingly for the test set
# unknown_regex = "/content/data/verification/unknown_dev/*" 
unknown_regex = "/content/data/verification/unknown_test/*"

# We load the images from known and unknown folders
unknown_images = [Image.open(p) for p in tqdm(sorted(glob.glob(unknown_regex)))]
known_images = [Image.open(p) for p in tqdm(sorted(glob.glob(known_regex)))]

# Why do you need only ToTensor() here?
transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()])

unknown_images = torch.stack([transforms(x) for x in unknown_images])
known_images  = torch.stack([transforms(y) for y in known_images ])
#Print your shapes here to understand what we have done

# You can use other similarity metrics like Euclidean Distance if you wish
similarity_metric = torch.nn.CosineSimilarity(dim= 1, eps= 1e-6) 

100%|██████████| 1000/1000 [00:00<00:00, 13207.29it/s]


In [40]:
def eval_verification(unknown_images, known_images, model, similarity, batch_size= config['batch_size'], mode='test'): 
    #change mode to test
    unknown_feats, known_feats = [], []

    batch_bar = tqdm(total=len(unknown_images)//batch_size, dynamic_ncols=True, position=0, leave=False, desc=mode)
    model.eval()

    # We load the images as batches for memory optimization and avoiding CUDA OOM errors
    for i in range(0, unknown_images.shape[0], batch_size):
        unknown_batch = unknown_images[i:i+batch_size] # Slice a given portion upto batch_size
        
        with torch.no_grad():
            unknown_feat = model(unknown_batch.float().to(device), return_feats=True) #Get features from model         
        unknown_feats.append(unknown_feat)
        batch_bar.update()
    
    batch_bar.close()
    
    batch_bar = tqdm(total=len(known_images)//batch_size, dynamic_ncols=True, position=0, leave=False, desc=mode)
    
    for i in range(0, known_images.shape[0], batch_size):
        known_batch = known_images[i:i+batch_size] 
        with torch.no_grad():
              known_feat = model(known_batch.float().to(device), return_feats=True)
          
        known_feats.append(known_feat)
        batch_bar.update()

    batch_bar.close()

    # Concatenate all the batches
    unknown_feats = torch.cat(unknown_feats, dim=0)
    known_feats = torch.cat(known_feats, dim=0)

    similarity_values = torch.stack([similarity(unknown_feats, known_feature) for known_feature in known_feats])
    # Print the inner list comprehension in a separate cell - what is really happening?

    predictions = similarity_values.argmax(0).cpu().numpy() #Why are we doing an argmax here?

    # Map argmax indices to identity strings
    pred_id_strings = [known_paths[i] for i in predictions]
    
    if mode == 'val':
      true_ids = pd.read_csv('/content/data/verification/dev_identities.csv')['label'].tolist()
      accuracy = accuracy_score(pred_id_strings, true_ids)
      print("Verification Accuracy = {}".format(accuracy))
    
    return pred_id_strings

In [41]:
pred_id_strings = eval_verification(unknown_images, known_images, model, similarity_metric, config['batch_size'], mode='test')

In [42]:
with open("verification_early_submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(pred_id_strings)):
        f.write("{},{}\n".format(i, pred_id_strings[i]))

Kaggel Submission

In [43]:
### Submit to kaggle competition using kaggle API
!kaggle competitions submit -c 11-785-f22-hw2p2-classification -f /content/classification_early_submission.csv -m "classification_Convnext"

100% 542k/542k [00:00<00:00, 2.80MB/s]
Successfully submitted to Face Recognition

In [44]:
### Submit to kaggle competition using kaggle API
!kaggle competitions submit -c 11-785-f22-hw2p2-verification -f /content/verification_early_submission.csv -m "verification_ConvNext"

100% 9.28k/9.28k [00:00<00:00, 49.6kB/s]
Successfully submitted to Face Verification